# Running on Euler Example

First set up the environment as described in `big-wave/Readme.md`, and then remember to update the path to the `rave-sim` directory and the desired simulation output directory below.

The `$SCRATCH` variable available on Euler is [intended for such data](https://scicomp.ethz.ch/wiki/Storage_systems#Personal_storage_.28everyone.29), ideally a subdirectory of `$SCRATCH` is used.

During an out-of-core simulation we read and write to the same files many times, so ideally we want to select a directory that is on the same node as where we run the code. For this purpose, SLURM provides the [local scratch directory](https://scicomp.ethz.ch/wiki/Using_local_scratch). We configure the required space via `--tmp={required_scratch_megs}` and then get the path through `$TMPDIR`.

We generate a `slurm-job.sh` script which can be launched with `sbatch`. The job uses a [job array](https://scicomp.ethz.ch/wiki/Job_arrays) with one task per source point.

In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
rave_sim_dir = Path("../")
simulations_dir = Path(os.environ.get("SCRATCH") or "/tmp") / "simulations"
simulations_dir.mkdir(exist_ok=True)

In [ ]:
sys.path.insert(0, str(rave_sim_dir / "big-wave"))
import multisim
import util

In [ ]:
config_dict = {
    "sim_params": {
        "N": 2**24,
        "dx": 3.1e-10,
        "z_detector": 1.77,
        "detector_size": 1e-3,
        "detector_pixel_size": 75e-6,
        "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
    },
    "use_disk_vector": True,
    "save_final_u_vectors": False,
    "multisource": {
        "type": "points",
        "energy_range": [46000.0, 47000.0],
        "x_range": [-1e-6, 1e-6],
        "z": 0.0,
        "nr_source_points": 12,
        "seed": 1,
    },
    "elements": [
        {
            "type": "grating",
            "pitch": 4.2e-6,
            "dc": [0.5, 0.5],
            "z_start": 0.1,
            "thickness": 140e-6,
            "nr_steps": 10,
            "x_positions": [0.0],
            "substrate_thickness": (370 - 140) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": ["Au", 19.32],
            "mat_substrate": ["Si", 2.34],
        },
        {
            "type": "grating",
            "pitch": 4.2e-6,
            "dc": [0.5, 0.5],
            "z_start": 0.918,
            "thickness": 59e-6,
            "nr_steps": 10,
            "x_positions": [0.0],
            "substrate_thickness": (200 - 59) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": None,
            "mat_substrate": ["Si", 2.34],
        },
        {
            "type": "grating",
            "pitch": 4.2e-6,
            "dc": [0.5, 0.5],
            "z_start": 1.736,
            "thickness": 154e-6,
            "nr_steps": 10,
            "x_positions": (np.arange(5) * 4.2e-6).tolist(),
            "substrate_thickness": (370 - 154) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": ["Au", 19.32],
            "mat_substrate": ["Si", 2.34],
        },
    ],
}

print("N: ", config_dict["sim_params"]["N"])

In [ ]:
sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)

In [ ]:
required_scratch_megs = int(config_dict["sim_params"]["N"] * 3 * 16 / 1024 / 1024) + 10


def job(sim_path: Path, nr_source_points: int) -> str:
    return f"""#!/usr/bin/env bash

#SBATCH --array=0-{nr_source_points - 1}
#SBATCH --ntasks=1
#SBATCH --time=3:55:00
#SBATCH --tmp={required_scratch_megs}
#SBATCH --mem-per-cpu=2G
#SBATCH --output="{str(sim_path / "slurm-out-%a.txt")}"
#SBATCH --error="{str(sim_path / "slurm-err-%a.txt")}"

if [ ! -f multisim.py ]; then
    echo "This script should be run in the big-wave directory"
    exit 1
fi

hostname
echo "jobid = $SLURM_JOB_ID"
echo "taskid = $SLURM_ARRAY_TASK_ID"
echo "tmpdir = $TMPDIR"

python3 <<EOF
from pathlib import Path
import os
import multisim
multisim.run_single_simulation(Path("{str(sim_path)}"), $SLURM_ARRAY_TASK_ID, Path(os.environ["TMPDIR"]))
EOF
"""


print("Run the following command in a shell from the big-wave directory.")
print("Make sure that an apropriate python environment activated.")
print()
p = sim_path / "slurm-job.sh"
with open(p, "w") as f:
    f.write(job(sim_path, config_dict["multisource"]["nr_source_points"]))

    print(f"sbatch {str(p)}")

## Analysis

Once the job array is finished, the outputs can be loaded using the `util.load_wavefronts_filtered` function.

In [ ]:
wavefronts = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=None)
wf = wavefronts[0][0]

In [ ]:
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size"])
plt.plot(detector_x, wf[0])